In [15]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [16]:
import galois
import numpy as np
import threading

np.set_printoptions(threshold=np.inf)

In [17]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [18]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [5]:
#provo co due matrici diagonali NON funziona
valori_diagonale = np.arange(1,127)
A = np.diag(valori_diagonale)

perm = np.random.permutation(126) #permutazione degli indici casuale
A_primo = A[perm, :]
A_primo = A_primo[:, perm]

A = field(A)
A_primo = field(A_primo)

In [19]:
A = field.Random((k, n_k))
#print(A)

In [6]:
id_mat = np.eye(126, dtype=int)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin1 = id_mat.copy()
matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin1 = field(matrice_bin1)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin2 = id_mat.copy()
matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin2 = field(matrice_bin2)

print(matrice_bin1)
print(matrice_bin2)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [7]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
#print(A_primo_intermedio)

In [8]:
A_primo = matrice_bin1 @ A @matrice_bin2
A_primo = field(A_primo)
#print(A_primo)

In [20]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

#creo il lock per evitare conglitti nell'aggiornamento
lock = threading.Lock()

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%127)

    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    with lock: #vado ad usare il lock
        somme_rig[indice_riga] = somma
        #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    with lock:
        primi_colonne[indice_colonna] = colonna[0]
        #print(primi_colonne)
        secondi_colonne[indice_colonna] = colonna[1]
        #print(secondi_colonne)
        terzi_colonne[indice_colonna] = colonna[2]
        quarti_colonne[indice_colonna] = colonna[3]



#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):
    threads_righe = []

    for indice_riga in range(matrice.shape[0]):
        t = threading.Thread(target = somma_riga, args = (matrice, indice_riga))
        threads_righe.append(t)
        t.start() #avvio il thread

    #aspetto il risultato di tutti i threads
    for t in threads_righe:
        t.join()

#eseguo in parallelo estrazione
def esegui_estrazione_parallelo(matrice):
    threads_colonne = []

    for indice_colonna in range(matrice.shape[1]):
        t = threading.Thread(target = estrai_da_colonna, args = (matrice, indice_colonna))
        threads_colonne.append(t)
        t.start() #avvio il thread

    #aspetto il risultato di tutti i threads
    for t in threads_colonne:
        t.join()

# Funzione per applicare una permutazione ciclica a un insieme di colonne
def permutazione_ciclica(matrice, blocco_dim):
    matrice_permutata = matrice.copy()
    
    # Eseguiamo una permutazione ciclica su tutte le colonne a gruppi di 'blocco_dim'
    for i in range(0, matrice.shape[1], blocco_dim):
        fine_blocco = min(i + blocco_dim, matrice.shape[1])
        # Cicla il blocco selezionato
        matrice_permutata[:, i:fine_blocco] = np.roll(matrice[:, i:fine_blocco], shift=1, axis=1)
    
    return field(matrice_permutata)


In [34]:
#fino a size 3 provato e funziona
while True:
    id_mat = np.eye(126, dtype=int)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin1 = id_mat.copy()
    matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin1 = field(matrice_bin1)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin2 = id_mat.copy()
    matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin2 = field(matrice_bin2)

    #print(matrice_bin1)
    #print(matrice_bin2)
    A_primo = matrice_bin1 @ A @ matrice_bin2
    A_primo = field(A_primo)
    #print(A_primo)"""
    somme_rig = [field(0)] * A_primo.shape[0]

    #lista per memorizzare i primi elementi della colonna
    primi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i secondi elemnti della colonna
    secondi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i terzi elementi della colonna
    terzi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i quarti elemnti della colonna
    quarti_colonne = [0] * A_primo.shape[1]

    #QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
    #PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

    colonne_finali = []
    esegui_somma_righe_parallelo(A_primo)
    #ordino in maniera crescente per valore gli indici
    indici_ordinati = np.argsort(somme_rig)
        #print(indici_ordinati)

        #applico gli swap
    A_canonica_righe = A_primo[indici_ordinati, :]
    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    #qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
    A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

    #print(A_CF)

    #devo fare lo stesso per la matrice A per dimostrare che sono CF

    colonne_finali = []

    #creo un thread per ogni colonna
    esegui_somma_righe_parallelo(A)

    #print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

    #ordino in maniera crescente i valori
    indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
    A_canonica_righe = A[indici_ordinati, :]

    #print(A_canonica_righe)

    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
    #print(A_CF1)

    zero = np.subtract(A_CF1,A_CF)
    zero = np.count_nonzero(zero)
    print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")
    if zero == 0:
        break


Le matrici differiscono per  252  elementi su 15876 (126*126)
Le matrici differiscono per  252  elementi su 15876 (126*126)
Le matrici differiscono per  0  elementi su 15876 (126*126)


In [35]:
print("Questa è A:", A)
print("\nQuesta è A': ", A_primo)
print("\nQuesta è la forma canonica: ", A_CF)

Questa è A: [[103   2   0  25 116  51  50  53 121  99  48  71  37  35  78  84   4  13
   49  41  62   5  45 109 114 113  35  63  21  48   5  22  71  59  84  16
   42  47  17  23  49  45   0  93 121  34  80  29  31  75  90 118  28 100
   75  99  76  41  34  14  99  35   5  96  57  85   0 121 123  94  24 126
   90  76  67  71  10  92 123   6   4  54 115  80  64 125  54 122 101  90
  103   5  63  94  77  30  49  79  12 101  95 118  93  12  21  81   5  92
   45  79 118  57  93  93  38  11  39  92  80  96  41  87  70  47  88  88]
 [ 37   5  49  21  96 111  88  10  72   4   8  78   2  39  51  87   7  96
   94 124   2 102 122 106  86  80  57 112 126 117  79  12  24   2 100  86
   39  79 100 105  72  64  43  92  77  38  43 121  23  54 110  81  85  76
  101   3  76  59  67 100   9  78  48  52  26  62  22  86  25 124  77 113
    8  53  85  82  38 113 104  53 111 117  15  32 103  58  89  52  36  37
   50  23  36 118  40  67  51  22  30  43  35  72 118  26  46  79  84  50
   29  28  49 114  76  29

In [25]:
import time

In [36]:
#Voglio vedere il tempo di esecuzione avendo già le matrici

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

colonne_finali = []
start_time = time.perf_counter()

esegui_somma_righe_parallelo(A_primo)
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]
#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

#print(A_CF)

#devo fare lo stesso per la matrice A per dimostrare che sono CF

colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)

for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
#print(A_CF1)
end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

zero = np.subtract(A_CF1,A_CF)
zero = np.count_nonzero(zero)
print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")

Durata esecuzione:  4037.3276729997087  ms
Le matrici differiscono per  0  elementi su 15876 (126*126)
